In [1]:
import pandas as pd
import random
import math
import copy
from collections import defaultdict, namedtuple

In [2]:
#2.8.0.0 - Manter a base de parâmetros
dias_semana = ["SEG", "TER", "QUA", "QUI", "SEX"]  #2.8.0.0
tempos = [1, 2, 3, 4, 5]                          #2.8.0.0

NUM_ITERACOES = 20000   #2.8.0.0 - Vamos seguir a mesma estrutura do ver.2.1 (múltiplos builds)
NUM_ITERACOES_SA = 500000 #2.8.0.0
TABU_ITERS = 1000        #2.8.0.0
TABU_TENURE = 1000        #2.8.0.0

In [3]:
#2.8.0.0 - Carregar arquivos
df_prof = pd.read_excel("professores.xlsx")  #2.8.0.0
df_demanda_raw = pd.read_excel("demanda.xlsx")  #2.8.0.0

In [4]:
#2.8.0.0 - CRIA DICIONÁRIO DE DISPONIBILIDADE (mesmo do ver2.1)
disponib = {}  #2.8.0.0
for i, row in df_prof.iterrows():  #2.8.0.0
    servidor = row["SERVIDOR"]
    turno    = row["TURNO"]
    if servidor not in disponib:
        disponib[servidor] = {}
    if turno not in disponib[servidor]:
        disponib[servidor][turno] = {}
    for d in dias_semana:
        if d not in disponib[servidor][turno]:
            disponib[servidor][turno][d] = {}
        for t in tempos:
            col = f"{d}{t}"
            if col in row:
                val = row[col]
                disponib[servidor][turno][d][t] = (val == 1)
            else:
                disponib[servidor][turno][d][t] = False

In [5]:
#2.8.0.0 - FILTRAR "APOIO" e montar demanda
df_demand = df_demanda_raw[df_demanda_raw["DISC"] != "APOIO"].copy()  #2.8.0.0
Pedido = namedtuple("Pedido", "prof turno nivel serie turma disc ch")  #2.8.0.0
demanda = []  #2.8.0.0
for i, row in df_demand.iterrows():  #2.8.0.0
    demanda.append(Pedido(
        prof  = row["SERVIDOR"],
        turno = row["TURNO"],
        nivel = row["NIVEL"],
        serie = str(row["SERIE/ANO"]),
        turma = str(row["TURMA"]),
        disc  = row["DISC"],
        ch    = int(row["CH"])
    ))

In [6]:
#2.8.0.1 - Criação do horário vazio
def cria_horario_vazio(demanda):
    horario = {}
    combos = set()
    for ped in demanda:
        combos.add((ped.turno, ped.nivel))
    for (tnr, niv) in combos:
        if tnr not in horario:
            horario[tnr] = {}
        if niv not in horario[tnr]:
            horario[tnr][niv] = {}
        for d in dias_semana:
            horario[tnr][niv][d] = {}
    for ped in demanda:
        for d in dias_semana:
            if (ped.serie, ped.turma) not in horario[ped.turno][ped.nivel][d]:
                horario[ped.turno][ped.nivel][d][(ped.serie, ped.turma)] = {}
    return horario

In [7]:
#2.8.0.1 - RESTRIÇÕES
def professor_disponivel(prof, turno, dia, tempo):
    if prof not in disponib:
        return False
    if turno not in disponib[prof]:
        return False
    return disponib[prof][turno][dia][tempo]

def professor_ocupado_esse_tempo(horario, prof, turno, niv, dia, tempo):
    for (_, dicT) in horario[turno][niv][dia].items():
        if tempo in dicT:
            (p, _) = dicT[tempo]
            if p == prof:
                return True
    return False

#2.8.0.1 - Aqui definimos a função que estava faltando, causando NameError
def professor_turma_ja_tem_aula(horario, turno, niv, dia, serie, turma, tempo):  #2.8.0.1
    """Retorna True se a turma (serie,turma) já tem uma aula naquele tempo."""
    return (tempo in horario[turno][niv][dia][(serie, turma)])   #2.8.0.1

def check_buraco(horario, ped, dia):
    dicT = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    if not dicT:
        return False
    tempos_ocup = sorted(dicT.keys())
    if tempos_ocup[0] != 1:
        return True
    maxi = max(tempos_ocup)
    for x in range(1, maxi+1):
        if x not in dicT:
            return True
    return False

def alocar_1tempo(horario, ped, dia, tempo):
    horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)][tempo] = (ped.prof, ped.disc)

def desalocar_1tempo(horario, ped, dia, tempo):
    dicTurma = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    if tempo in dicTurma:
        del dicTurma[tempo]

def check_professor_gap(horario, prof, turno, dia):
    allocated_times = []
    for niv in horario[turno]:
        for (_, turma), dic in horario[turno][niv][dia].items():
            for t, (p, _) in dic.items():
                if p == prof:
                    allocated_times.append(t)
    if len(allocated_times) <= 1:
        return False
    allocated_times = sorted(allocated_times)
    for i in range(len(allocated_times) - 1):
        if allocated_times[i+1] - allocated_times[i] > 1:
            return True
    return False

def professor_turma_mesmo_dia(horario, ped, dia): #2.8.0.1 Correção
    """
    Verifica se o professor 'ped.prof' já está alocado nessa turma (ped.serie, ped.turma)
    neste mesmo dia 'dia'. Se sim, retorna True para indicar que não pode alocar novamente.
    """
    dicTurma = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    for tm, (prof, _) in dicTurma.items():
        if prof == ped.prof:
            return True
    return False

def pode_alocar(horario, ped, dia, tempo):
    if not professor_disponivel(ped.prof, ped.turno, dia, tempo):
        return False
    if professor_ocupado_esse_tempo(horario, ped.prof, ped.turno, ped.nivel, dia, tempo):
        return False
    #2.8.0.1 - antes de alocar, checar se a turma já tem algo no mesmo tempo
    if professor_turma_ja_tem_aula(horario, ped.turno, ped.nivel, dia, ped.serie, ped.turma, tempo):
        return False

    #2.8.0.1 - Tenta alocar e ver se gera buraco
    alocar_1tempo(horario, ped, dia, tempo)
    if check_buraco(horario, ped, dia):
        desalocar_1tempo(horario, ped, dia, tempo)
        return False
    if check_professor_gap(horario, ped.prof, ped.turno, dia):
        desalocar_1tempo(horario, ped, dia, tempo)
        return False
    desalocar_1tempo(horario, ped, dia, tempo)
    return True

def check_professor_gap(horario, prof, turno, dia):
    allocated_times = []
    for niv in horario[turno]:
        for (_, turma), dic in horario[turno][niv][dia].items():
            for t, (p, _) in dic.items():
                if p == prof:
                    allocated_times.append(t)
    if len(allocated_times) <= 1:
        return False
    allocated_times = sorted(allocated_times)
    for i in range(len(allocated_times) - 1):
        if allocated_times[i+1] - allocated_times[i] > 1:
            return True
    return False

def pode_alocar(horario, ped, dia, tempo):
    if not professor_disponivel(ped.prof, ped.turno, dia, tempo):
        return False
    if professor_ocupado_esse_tempo(horario, ped.prof, ped.turno, ped.nivel, dia, tempo):
        return False
    if professor_turma_ja_tem_aula(horario, ped.turno, ped.nivel, dia, ped.serie, ped.turma, tempo):
        return False

    if professor_turma_mesmo_dia(horario, ped, dia): #2.8.0.1 Correção
    # Professor já foi alocado nessa turma nesse dia, rejeita
        return False

    #2.8.0.0 - Temporariamente alocar e checar buraco
    alocar_1tempo(horario, ped, dia, tempo)
    if check_buraco(horario, ped, dia):
        desalocar_1tempo(horario, ped, dia, tempo)
        return False
    #2.8.0.0 - Check gap professor
    if check_professor_gap(horario, ped.prof, ped.turno, dia):
        desalocar_1tempo(horario, ped, dia, tempo)
        return False
    desalocar_1tempo(horario, ped, dia, tempo)
    return True

In [8]:
#2.8.0.0 - SCORE E PENALIDADES
def conta_alocacoes(horario):
    used = 0
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (_, turma), dic in horario[tnr][niv][d].items():
                    used += len(dic)
    return used

def total_slots_ideal(demanda):
    turmas = set((ped.serie, ped.turma) for ped in demanda)
    return len(turmas) * 25

def total_gaps(horario): #2.8.0.3
    gaps = 0
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    if not dic:
                        # Se não há nenhum slot preenchido nesse dia para a turma,
                        # penaliza com 1 (mesmo peso que um gap)
                        gaps += 1
                    else:
                        tempos_ocup = sorted(dic.keys())
                        maxi = max(tempos_ocup)
                        for t in range(1, maxi + 1):
                            if t not in dic:
                                gaps += 1
    return gaps

def calcula_penalidades(horario, demanda):
    peso_alocacao_extra = 50
    peso_gap_prof = 0
    penalidade_total = 0

    for ped in demanda:
        for d in dias_semana:
            contagem = 0
            for niv in horario[ped.turno]:
                turma_aloc = horario[ped.turno][niv][d].get((ped.serie, ped.turma), {})
                for t, (p, _) in turma_aloc.items():
                    if p == ped.prof:
                        contagem += 1
            if contagem > 1:
                penalidade_total += peso_alocacao_extra * (contagem - 1)

    for prof in disponib.keys():
        for turno in disponib[prof]:
            for d in dias_semana:
                allocated_times = []
                for niv in horario[turno]:
                    for (_, turma), dic in horario[turno][niv][d].items():
                        for t, (p, _) in dic.items():
                            if p == prof:
                                allocated_times.append(t)
                if len(allocated_times) > 1:
                    allocated_times = sorted(allocated_times)
                    for i in range(len(allocated_times) - 1):
                        if allocated_times[i + 1] - allocated_times[i] > 1:
                            penalidade_total += peso_gap_prof
    return penalidade_total

def score_solucao_com_penalidades(horario, demanda):
    usados = conta_alocacoes(horario)
    slots = total_slots_ideal(demanda)
    nao_aloc = sum(p.ch for p in demanda) - usados
    gaps_turma = total_gaps(horario)
    peso_gap_turma = 1000
    penalidade_soft = calcula_penalidades(horario, demanda)
    score = usados - peso_gap_turma * gaps_turma - 200 * nao_aloc - penalidade_soft
    feats = {
        "usados": usados,
        "slots": slots,
        "gaps": gaps_turma,
        "nao_alocados": nao_aloc,
        "penalidades_soft": penalidade_soft,
        "percent_filled": (usados / slots) * 100 if slots else 0
    }
    return score, feats

In [9]:
#2.8.0.0 - HEURÍSTICA DE ALOCAÇÃO (do ver2.1) adaptada
def alocar_demanda(horario, demanda):  #2.8.0.0
    """
    Inspirado em ver2.1: sortear a ordem da demanda, para cada pedido (prof, disc, CH),
    distribuir ch tempos em dias/tempos aleatórios, respeitando restrições.
    Retorna lista de (pedido, ch_nao_alocado).
    """  #2.8.0.0
    dem_local = list(demanda)
    random.shuffle(dem_local)  #2.8.0.0

    nao_alocados = []
    for ped in dem_local:
        ch_rest = ped.ch
        #2.8.0.0 - Montamos uma lista de (dia,tempo) embaralhada
        candidatos_dt = []
        for d in dias_semana:
            for t in tempos:
                candidatos_dt.append((d, t))
        random.shuffle(candidatos_dt)

        for (d, t) in candidatos_dt:
            if ch_rest <= 0:
                break
            if pode_alocar(horario, ped, d, t):
                alocar_1tempo(horario, ped, d, t)
                ch_rest -= 1

        if ch_rest > 0:
            nao_alocados.append((ped, ch_rest))
    return nao_alocados

def constroi_solucao(demanda):  #2.8.0.0
    """Fazemos o create horario vazio e chamamos alocar_demanda."""
    horario_temp = cria_horario_vazio(demanda)
    nao_aloc = alocar_demanda(horario_temp, demanda)
    return horario_temp, nao_aloc

In [10]:
#2.8.0.0 - MULTIPLAS ITERACOES
melhor_horario = None  #2.8.0.0
melhor_nao_aloc = None  #2.8.0.0
melhor_score = -999999  #2.8.0.0

for i in range(NUM_ITERACOES):  #2.8.0.0
    hor_cand, nao_cand = constroi_solucao(demanda)  #2.8.0.0
    sc, ft = score_solucao_com_penalidades(hor_cand, demanda)  #2.8.0.0
    if sc > melhor_score:  #2.8.0.0
        melhor_score = sc  #2.8.0.0
        melhor_horario = hor_cand  #2.8.0.0
        melhor_nao_aloc = nao_cand  #2.8.0.0

print(f"#2.8.0.0 - Melhor solução inicial (heurística ver2.1) => score = {melhor_score}")  #2.8.0.0

#2.8.0.0 - Melhor solução inicial (heurística ver2.1) => score = -41670


In [11]:
# CÓDIGO PARA CALCULAR A PORCENTAGEM DE SLOTS PREENCHIDOS NA SOLUÇÃO INICIAL

usados = conta_alocacoes(melhor_horario)  # Número de slots preenchidos na solução inicial
total_slots = total_slots_ideal(demanda)   # Total de slots disponíveis (por turma)

if total_slots > 0:
    perc_slots = 100 * usados / total_slots
else:
    perc_slots = 0

print(f"Porcentagem de slots preenchidos na solução inicial: {perc_slots:.2f}%")

Porcentagem de slots preenchidos na solução inicial: 66.00%


In [12]:
#2.8.0.0 - FASE SA
def remover_tempo_aleatorio(horario):
    alocados = []
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    for tm, (prof, disc) in dic.items():
                        alocados.append((tnr, niv, d, ser, tur, tm, prof, disc))
    if not alocados:
        return None
    (tnr, niv, d, ser, tur, tm, prof, disc) = random.choice(alocados)
    del horario[tnr][niv][d][(ser, tur)][tm]
    pedido_falso = Pedido(prof=prof, turno=tnr, nivel=niv, serie=ser, turma=tur, disc=disc, ch=1)
    return pedido_falso, d, tm

def simulated_annealing(horario, nao_alocados, demanda, num_iter=NUM_ITERACOES_SA, T0=500.0, alpha=0.999):
    current_hor = copy.deepcopy(horario)
    current_nao = copy.deepcopy(nao_alocados)
    current_score, _ = score_solucao_com_penalidades(current_hor, demanda)

    best_hor = copy.deepcopy(current_hor)
    best_nao = copy.deepcopy(current_nao)
    best_score = current_score

    T = T0
    for i in range(num_iter):
        new_hor = copy.deepcopy(current_hor)
        new_nao = copy.deepcopy(current_nao)

        if random.random() < 0.5:
            remov = remover_tempo_aleatorio(new_hor)
            if remov is not None:
                (pedido_falso, d_rem, t_rem) = remov
                new_nao.append((pedido_falso, 1))

        if new_nao:
            idx = random.choice(range(len(new_nao)))
            ped, ch_rest = new_nao[idx]
            allocated = False
            dias_rand = copy.deepcopy(dias_semana)
            random.shuffle(dias_rand)
            tempos_rand = copy.deepcopy(tempos)
            random.shuffle(tempos_rand)

            for d in dias_rand:
                if ch_rest <= 0:
                    break
                for t in tempos_rand:
                    if ch_rest <= 0:
                        break
                    if pode_alocar(new_hor, ped, d, t):
                        alocar_1tempo(new_hor, ped, d, t)
                        ch_rest -= 1
                        allocated = True
                        break
                if allocated:
                    break
            if ch_rest == 0:
                new_nao.pop(idx)
            else:
                new_nao[idx] = (ped, ch_rest)

        new_score, _ = score_solucao_com_penalidades(new_hor, demanda)
        delta = new_score - current_score
        if delta > 0 or random.random() < math.exp(delta / T):
            current_hor = new_hor
            current_nao = new_nao
            current_score = new_score
            if new_score > best_score:
                best_hor = copy.deepcopy(new_hor)
                best_nao = copy.deepcopy(new_nao)
                best_score = new_score
        T *= alpha
    return best_hor, best_nao

hor_sa, nao_sa = simulated_annealing(melhor_horario, melhor_nao_aloc, demanda)  #2.8.0.0
score_sa, feats_sa = score_solucao_com_penalidades(hor_sa, demanda)  #2.8.0.0
print(f"#2.8.0.0 - Após SA: score = {score_sa}, fill = {feats_sa['percent_filled']:.2f}%")  #2.8.0.0

#2.8.0.0 - Após SA: score = -11535, fill = 93.00%


In [13]:
#2.8.0.0 - TABU SEARCH (igual ver2.7.5.5)
def gera_vizinhos_tabu(horario, nao_alocados, demanda):  #2.8.0.0
    vizinhos = []
    alocados = []
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    for tm, (prof, disc) in dic.items():
                        alocados.append((tnr, niv, d, ser, tur, tm, prof, disc))
    if not alocados:
        return vizinhos
    random.shuffle(alocados)
    # 2.8.0.2: Aumentado o número de candidatos de 5 para 10
    alocados = alocados[:10]

    for (tnr, niv, d, ser, tur, tm, prof, disc) in alocados:
        # Operação de Remoção/Realocação
        new_hor = copy.deepcopy(horario)
        new_nao = copy.deepcopy(nao_alocados)
        del new_hor[tnr][niv][d][(ser, tur)][tm]
        pedido_falso = Pedido(prof=prof, turno=tnr, nivel=niv, serie=ser, turma=tur, disc=disc, ch=1)
        new_nao.append((pedido_falso, 1))
        # 2.8.0.2: Tenta realocar o pedido removido em novos slots usando ordem fixa
        for dd in dias_semana:  # 2.8.0.2
            for ttx in tempos:  # 2.8.0.2
                if pode_alocar(new_hor, pedido_falso, dd, ttx):
                    alocar_1tempo(new_hor, pedido_falso, dd, ttx)
                    break
            else:
                continue
            break
        vizinhos.append((new_hor, new_nao))

        # Operador de Swap: troca a alocação do candidato com outra alocação aleatória
        if len(alocados) > 1:
            swap_candidates = [c for c in alocados if c != (tnr, niv, d, ser, tur, tm, prof, disc)]
            if swap_candidates:
                swap_candidate = random.choice(swap_candidates)
                tnr2, niv2, d2, ser2, tur2, tm2, prof2, disc2 = swap_candidate
                new_hor_swap = copy.deepcopy(horario)
                new_nao_swap = copy.deepcopy(nao_alocados)
                # Remover as duas alocações
                del new_hor_swap[tnr][niv][d][(ser, tur)][tm]
                del new_hor_swap[tnr2][niv2][d2][(ser2, tur2)][tm2]
                # Tentar swap: realocar a alocação do primeiro no slot do segundo e vice-versa
                pedido1 = Pedido(prof=prof, turno=tnr, nivel=niv, serie=ser, turma=tur, disc=disc, ch=1)
                pedido2 = Pedido(prof=prof2, turno=tnr2, nivel=niv2, serie=ser2, turma=tur2, disc=disc2, ch=1)
                if pode_alocar(new_hor_swap, pedido1, d2, tm2) and pode_alocar(new_hor_swap, pedido2, d, tm):
                    alocar_1tempo(new_hor_swap, pedido1, d2, tm2)
                    alocar_1tempo(new_hor_swap, pedido2, d, tm)
                    vizinhos.append((new_hor_swap, new_nao_swap))
    return vizinhos

def tabu_search(horario, nao_alocados, demanda, iters=TABU_ITERS, tenure=TABU_TENURE):  #2.8.0.0
    current_hor = copy.deepcopy(horario)
    current_nao = copy.deepcopy(nao_alocados)
    current_score, _ = score_solucao_com_penalidades(current_hor, demanda)

    best_hor = copy.deepcopy(current_hor)
    best_nao = copy.deepcopy(current_nao)
    best_score = current_score

    def state_hash(h, n):
        items = []
        for tnr in h:
            for niv in h[tnr]:
                for d in dias_semana:
                    for (ser, tur), dic in h[tnr][niv][d].items():
                        for tm, (pf, ds) in dic.items():
                            items.append((tnr, niv, d, ser, tur, tm, pf, ds))
        items.sort()
        return str(items) + str(n)

    tabu_list = {}
    itc = 0
    while itc < iters:
        itc += 1
        neighbors = gera_vizinhos_tabu(current_hor, current_nao, demanda)
        best_local = None
        best_local_score = -999999
        for (h2, nao2) in neighbors:
            shash = state_hash(h2, nao2)
            s2, _ = score_solucao_com_penalidades(h2, demanda)
            if shash not in tabu_list or s2 > best_score:
                if s2 > best_local_score:
                    best_local = (h2, nao2)
                    best_local_score = s2

        if best_local is None:
            break

        current_hor, current_nao = best_local
        current_score = best_local_score
        if current_score > best_score:
            best_score = current_score
            best_hor = copy.deepcopy(current_hor)
            best_nao = copy.deepcopy(current_nao)

        sh = state_hash(current_hor, current_nao)
        tabu_list[sh] = itc + tenure
        for k in list(tabu_list.keys()):
            if tabu_list[k] <= itc:
                del tabu_list[k]

    return best_hor, best_nao

hor_tabu, nao_tabu = tabu_search(hor_sa, nao_sa, demanda)  #2.8.0.0
score_final, feats_final = score_solucao_com_penalidades(hor_tabu, demanda)  #2.8.0.0
print(f"#2.8.0.0 - Após TabuSearch => score_final={score_final}, fill={feats_final['percent_filled']:.2f}%")  #2.8.0.0

hor_final = hor_tabu  #2.8.0.0
nao_aloc_final = nao_tabu  #2.8.0.0

#2.8.0.0 - Após TabuSearch => score_final=-15153, fill=89.40%


In [14]:
#2.8.0.0 - Resumo final CH
total_CH_por_servidor = {}
for ped in demanda:
    total_CH_por_servidor[ped.prof] = total_CH_por_servidor.get(ped.prof, 0) + ped.ch

CH_alocada_por_servidor = {}
for tnr in hor_final:
    for niv in hor_final[tnr]:
        for d in dias_semana:
            for (serie, turma), dic in hor_final[tnr][niv][d].items():
                for t in dic:
                    prof, _ = dic[t]
                    CH_alocada_por_servidor[prof] = CH_alocada_por_servidor.get(prof, 0) + 1

resumo_CH = []
for prof in total_CH_por_servidor:
    demanded = total_CH_por_servidor[prof]
    allocated = CH_alocada_por_servidor.get(prof, 0)
    left = demanded - allocated
    resumo_CH.append({
        "Servidor": prof,
        "Total CH Demandado": demanded,
        "CH Alocada": allocated,
        "CH Não Alocada": left
    })

import pandas as pd
df_resumo_CH = pd.DataFrame(resumo_CH)
total_linha = {
    "Servidor": "TOTAL",
    "Total CH Demandado": df_resumo_CH["Total CH Demandado"].sum(),
    "CH Alocada": df_resumo_CH["CH Alocada"].sum(),
    "CH Não Alocada": df_resumo_CH["CH Não Alocada"].sum()
}
df_total = pd.DataFrame([total_linha])
df_resumo_CH = pd.concat([df_resumo_CH, df_total], ignore_index=True)

print("#2.8.0.0 - Resumo Alocação de CH:")
print(df_resumo_CH)

df_features = pd.DataFrame([{"Score_Final": score_final, **feats_final}])
df_features.to_excel("FEATURES_SOLUCOES.xlsx", index=False)

#2.8.0.0 - Resumo Alocação de CH:
                  Servidor  Total CH Demandado  CH Alocada  CH Não Alocada
0   AMARILDO SENA DE FARIA                  15          13               2
1   ANDREA CRISTINA NASCIM                  18          16               2
2   AVONEIDE DA SILVA MEND                  19          17               2
3   AYRTON LUCAS LIMA TELE                  16          16               0
4   CHRISTINA SIMAS CORREA                  16          14               2
5   CLAUDENOR DE SOUZA PIE                  16          15               1
6      DANIEL ESTEVES RAID                  15          14               1
7   EDIVALDO DE SOUZA OLIV                  16          14               2
8    ELIZABETH DE OLIVEIRA                  16          13               3
9                    ERIKA                   7           6               1
10  GLAURIA GLEICE GAMA DO                  16          13               3
11  JOELMA DA SILVA ARAUJO                  16          14        

In [15]:
#2.8.0.0 - EXPORTAÇÃO
def gera_df_por_turma(horario):
    from collections import defaultdict
    colunas_por_tn = defaultdict(set)
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    colunas_por_tn[(tnr, niv)].add(f"{ser}-{tur}")
    dfs_por_aba = {}
    for tnr in horario:
        for niv in horario[tnr]:
            turmas_cols = sorted(list(colunas_por_tn[(tnr, niv)]))
            linhas = []
            for d in dias_semana:
                for tm in tempos:
                    row_dict = {"TURNO": tnr, "NIVEL": niv, "DIA": d, "TEMPO": tm}
                    for col in turmas_cols:
                        row_dict[col] = ""
                    for (ser, turma), dic in horario[tnr][niv][d].items():
                        if tm in dic:
                            (p, ds) = dic[tm]
                            colname = f"{ser}-{turma}"
                            row_dict[colname] = f"{p}+{ds}"
                    linhas.append(row_dict)
            df_aba = pd.DataFrame(linhas)
            dfs_por_aba[f"{tnr}_{niv}"] = df_aba
    return dfs_por_aba

def gera_df_por_professor(horario):
    from collections import defaultdict
    prof_data = defaultdict(lambda: defaultdict(list))
    prof_cols = defaultdict(set)
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    for tm, (p, ds) in dic.items():
                        col = f"{ser}-{tur}({ds})"
                        prof_cols[p].add(col)
                        prof_data[p][(d, tm)].append((tnr, niv, ser, tur, ds))
    dfs_por_prof = {}
    for p in prof_data:
        cols = sorted(list(prof_cols[p]))
        linhas = []
        for d in dias_semana:
            for tm in tempos:
                row_dict = {"PROFESSOR": p, "DIA": d, "TEMPO": tm}
                for c in cols:
                    row_dict[c] = ""
                if (d, tm) in prof_data[p]:
                    for (tnr, niv, ser, tur, ds) in prof_data[p][(d, tm)]:
                        cname = f"{ser}-{tur}({ds})"
                        row_dict[cname] = f"{tnr}-{niv}"
                linhas.append(row_dict)
        df_p = pd.DataFrame(linhas)
        dfs_por_prof[p] = df_p
    return dfs_por_prof

dfs_turma = gera_df_por_turma(hor_final)  #2.8.0.0
dfs_prof = gera_df_por_professor(hor_final)  #2.8.0.0

with pd.ExcelWriter("QUADRO_HORÁRIOS_TURMAS.xlsx") as writer:  #2.8.0.0
    for aba, df_ in dfs_turma.items():
        df_.to_excel(writer, sheet_name=aba[:31], index=False)

with pd.ExcelWriter("QUADRO_HORÁRIOS_PROFESSORES.xlsx") as writer:  #2.8.0.0
    for p, df_ in dfs_prof.items():
        sheetname = p[:31]
        df_.to_excel(writer, sheet_name=sheetname, index=False)

vazios_detalhe = []  #2.8.0.0
for tnr in hor_final:
    for niv in hor_final[tnr]:
        for d in dias_semana:
            for (ser, tur), dic in hor_final[tnr][niv][d].items():
                for tm in tempos:
                    if tm not in dic:
                        vazios_detalhe.append({
                            "TURNO": tnr,
                            "NIVEL": niv,
                            "DIA": d,
                            "SERIE": ser,
                            "TURMA": tur,
                            "TEMPO": tm
                        })

df_vaz = pd.DataFrame(vazios_detalhe)  #2.8.0.0
df_vaz.to_excel("NAO_ALOCADOS_TEMPOS_VAZIOS.xlsx", index=False)  #2.8.0.0

print("#2.8.0.0 - Arquivos gerados:")
print(" - QUADRO_HORÁRIOS_TURMAS.xlsx (abas por Turno_Nível)")
print(" - QUADRO_HORÁRIOS_PROFESSORES.xlsx (abas por professor)")
print(" - NAO_ALOCADOS_TEMPOS_VAZIOS.xlsx (slots vazios)")
print(" - FEATURES_SOLUCOES.xlsx (características da solução final)")


#2.8.0.0 - Arquivos gerados:
 - QUADRO_HORÁRIOS_TURMAS.xlsx (abas por Turno_Nível)
 - QUADRO_HORÁRIOS_PROFESSORES.xlsx (abas por professor)
 - NAO_ALOCADOS_TEMPOS_VAZIOS.xlsx (slots vazios)
 - FEATURES_SOLUCOES.xlsx (características da solução final)


In [16]:
# Código para gerar resumo de CH não alocada por professor em relação a série/turma

# Primeiro, calcular a carga horária demandada por professor/serie/turma
demanded_by_prof_turma = {}
for ped in demanda:
    key = (ped.prof, ped.serie, ped.turma)
    demanded_by_prof_turma[key] = demanded_by_prof_turma.get(key, 0) + ped.ch

# Em seguida, calcular a CH alocada por professor/serie/turma na solução final
allocated_by_prof_turma = {}
# Usando a solução final, por exemplo, 'hor_final'
for tnr in hor_final:
    for niv in hor_final[tnr]:
        for d in dias_semana:
            for (serie, turma), dic in hor_final[tnr][niv][d].items():
                for t in dic:
                    prof, _ = dic[t]
                    key = (prof, serie, turma)
                    allocated_by_prof_turma[key] = allocated_by_prof_turma.get(key, 0) + 1

# Monta o resumo comparando a CH demandada com a CH alocada para cada professor/serie/turma
summary_list = []
for key, demanded in demanded_by_prof_turma.items():
    allocated = allocated_by_prof_turma.get(key, 0)
    deficit = demanded - allocated
    summary_list.append({
        "Professor": key[0],
        "Série": key[1],
        "Turma": key[2],
        "CH Demandado": demanded,
        "CH Alocada": allocated,
        "CH Não Alocada": deficit
    })

# Cria o DataFrame e exporta para Excel
df_summary = pd.DataFrame(summary_list)
print("Resumo de CH não alocada por professor/serie/turma:")
print(df_summary)
df_summary.to_excel("CH_NAO_ALOCADA_POR_PROF_TURMA.xlsx", index=False)

Resumo de CH não alocada por professor/serie/turma:
                  Professor Série Turma  CH Demandado  CH Alocada  \
0    AMARILDO SENA DE FARIA     3     5             2           2   
1    AMARILDO SENA DE FARIA     3     4             2           2   
2    AMARILDO SENA DE FARIA     3     3             2           2   
3    AMARILDO SENA DE FARIA     3     2             2           1   
4    AMARILDO SENA DE FARIA     3     1             2           2   
..                      ...   ...   ...           ...         ...   
254                VAGA 02      1     1             1           0   
255  WELIGTHON JOSE MARTINS     2     4             4           3   
256  WELIGTHON JOSE MARTINS     2     7             4           4   
257  WELIGTHON JOSE MARTINS     2     6             4           4   
258  WELIGTHON JOSE MARTINS     2     5             4           3   

     CH Não Alocada  
0                 0  
1                 0  
2                 0  
3                 1  
4        